# 셀레니움을 이용한 자동화

In [4]:
# 1-1. 라이브러리 선언

# 셀레니움 라이브러리
from selenium import webdriver
from selenium.webdriver.common.keys import Keys # 텍스트형식 클릭을 하려면 필요함
import datetime
import time

# 데이터 추출 라이브러리
from bs4 import BeautifulSoup #태그를 보기 좋게 뜸
import requests # 웹페이지 접속요청
import pandas as pd

# 메일전송 라이브러리
import os
import smtplib
 
from email import encoders
from email.utils import formataddr
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [13]:
# 1-2. 함수선언 (스크롤다운)
def doScrollDown(whileSeconds):
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=whileSeconds)
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(1)
        if datetime.datetime.now() > end:
            break

rowList = []
output = []
            
# 2. 웹브라우저 설정
options = webdriver.ChromeOptions()

# options.add_argument("window-size = 1600*900") # 테스트용 코드
options.add_argument("headless") # 헤드리스 코드

driverLoc = "./addon/chromedriver/chromedriver.exe"

driver = webdriver.Chrome(executable_path = driverLoc, options = options)

# 페이지의 완전한 로딩을 위해 3초간 기다림
driver.implicitly_wait(3)

# 3. 브라우저 열기
targetUrl = "https://shopping.naver.com/"
driver.get(targetUrl)

# 4-1. 브라우저 내 액션 (클릭)
# xpath 이용
catBtxpath = '//*[@id="rootCategoryList"]/button'
catBtn = driver.find_element_by_xpath(catBtxpath)
catBtn.click()

targetcatBtxpath = '//*[@id="categoryLink50000169"]'
targetcatBtn = driver.find_element_by_xpath(targetcatBtxpath)
targetcatBtn.click()

# 해당 페이지만 이동하며 10개의 페이지 정보 수집
for page in range(1, 10):
    time.sleep(3)
    doScrollDown(3)

# 5. 페이지 url 가져오기
    targetUrl = driver.current_url

## 로딩 완료한 페이지 정보 수집 ##

# 1. 타겟 url의 html 소스 불러오기
    resp = requests.get(url=targetUrl)
    resp.encoding = "utf-8"
    html = resp.text
    htmlObj = BeautifulSoup(html, "html.parser")

    tableTaginfo = htmlObj.find(name = "ul", attrs = {"class" : "list_basis"})

# 2. 데이터를 축적할 리스트 및 필요태그 선언

    columnList = []

    prdPriceinfo = tableTaginfo.findAll(name = "span", attrs = {"class" : "price_num__2WUXn"})
    prdCatinfo = tableTaginfo.findAll(name = "div", attrs = {"class" : "basicList_depth__2QIie"})
    prdNameinfo = tableTaginfo.findAll(name="div", attrs = {"class" : "basicList_title__3P9Q7"})

    for i in range (0, len(prdNameinfo)):

        refinedCat = prdCatinfo[i].text
        refinedName = prdNameinfo[i].text
        refinedPrice = prdPriceinfo[i].text

        # 카테고리 명 중복문구를 제거하여 필요한 정보 추출
        distinctCat = refinedCat.replace("패션의류남성의류", "")

        columnList.append(refinedName)
        columnList.append(refinedPrice)
        columnList.append(distinctCat)

        rowList.append(columnList)
        columnList = []
        
        output = pd.DataFrame(rowList, columns = ['productname', 'productprice', 'productcategory'])

    nextLinkPath = '//*[@id="__next"]/div/div[2]/div/div[3]/div[1]/div[3]/div/a[' + str(page) + ']'
    nextBtn = driver.find_element_by_xpath(nextLinkPath)
    nextBtn.click()
    
output.to_csv('sample.csv')

In [5]:
output

,productname,productprice,productcategory
0,17수 순면 티셔츠 1장부터 인쇄 단체티 맞춤,"5,900원",유니폼/단체복
1,"20수,30수,45수,기능성 쿨원단 반팔/칠부 주방조리사복 쉐프웨어 조리복","21,000원",유니폼/단체복
2,맥커스 와이드 머슬핏 반팔 티셔츠,"19,600원",티셔츠
3,[에두아르도 자체제작][5팩 세트]세미오버 반팔 티셔츠 스탠다드세트,"66,900원",티셔츠
4,타미힐피거 반팔 티셔츠 남자 여자 면티 T3139,"14,112원",티셔츠
5,티셔츠주문제작 라운드 반팔,"3,900원",티셔츠
6,[28-42] 남자 봄 청바지 스판 데님 연청 진청 블랙진 테이퍼드 일자 빅사이즈 58종,"15,900원",청바지
7,ROKA 로카티 반팔티 긴팔티 군인 군대티셔츠,"8,500원",티셔츠
8,해외폴로 랄프로렌 남성 클래식핏 린넨 셔츠 Classic Fit Linen Shirt,"119,900원",셔츠/남방
9,칼하트 K87 남성 포켓 티셔츠 17종 택1,"15,300원",티셔츠


In [10]:
import smtplib
from email.mime.text import MIMEText

session = smtplib.SMTP('smtp.gmail.com', 587)

# tls 보안 시작 및 로그인
session.ehlo()
session.starttls()
session.login('bxob1040@gmail.com', 'iwuzjqfaqughzari')

# 송수신자 입력
from_addr = formataddr(('Google acctest', 'bxob1040@gmail.com'))
to_addr = formataddr(('Naver acctest', 'bxob1040@naver.com'))

# 메세지 콘텐츠 설정
msg = MIMEMultipart("mixed")

# 메세지 송/수신 옵션
msg.set_charset('utf-8')
msg['From'] = 'testmail.'
msg['To'] = 'testmail.'
senddate = datetime.datetime.now()
msg['Subject'] = 'crawlres_' + str(senddate)

# 메세지 콘텐츠 - 내용
body = '''
<h2>위 일자에 해당하는 크로울링 내용입니다.</h1>
'''
bodyPart = MIMEText(body, 'html', 'utf-8')
msg.attach( bodyPart )

# 메일 콘텐츠 - 첨부파일
attachments = [
    os.path.join( os.getcwd(), 'project_navershopping_man', 'sample.csv' )]
 
for attachment in attachments:
    attach_binary = MIMEBase("application", "octect-stream")
    try:
        binary = open(attachment, "rb").read() # read file to bytes
 
        attach_binary.set_payload( binary )
        encoders.encode_base64( attach_binary ) # Content-Transfer-Encoding: base64
            
        filename = os.path.basename( attachment )
        attach_binary.add_header("Content-Disposition", 'attachment', filename=('utf-8', '', filename))
            
        msg.attach( attach_binary )
    except Exception as e:
        print( e )

# 메일 보내기
session.sendmail("bxob1040@gmail.com", "bxob1040@naver.com", msg.as_string())

# 세션 종료
session.quit()

(221, b'2.0.0 closing connection z17sm8517008pfe.181 - gsmtp')